### This notebook contains the code used to analyze the experimental data and compute the results reported in the paper "Inspiration through Observation: Demonstrating the Influence of Automatically Generated Text on Human Creative Writing".

Library requirements: pandas, numpy, scikit-learn, transformers

In [1]:
import pandas
import os
import numpy
import sys
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity

pandas.set_option('display.max_columns', None)
pandas.set_option("max_colwidth", -1)

vectorizer = pipeline('feature-extraction')

### Load and visualize data

In [2]:
'''Authoring blocks resulting from human authoring task'''

authoring_blocks = pandas.read_json("authoring_blocks.json")
authoring_blocks

,Session_ID,Block_ID,Difficulty_Level,Input_Words,Pre_Time_Elapsed,Post_Time_Elapsed,Pre_User_Sentences,Post_User_Sentences,Generated_Sentences
0,6f272542-6353-4256-8d79-a2ecf9e950c9.txt,1226,easy,the old fashioned,42.728305,71.056639,"[The old fashioned prince arrived at the castle. , The old man arrived at the old fashioned restaurant.]","[I saw the old fashioned grandfather clock in the window display earlier today. , The old fashioned bartender told everyone that he normally does not serve drinks after 9pm.]","[But the old fashioned life is short lived., The Old fashioned look of the project can't be much worse., In fact, the town was an old fashioned one., You can make the same mistakes as the old fashioned way., The former Old fashioned, but the latter is more polite.]"
1,6f272542-6353-4256-8d79-a2ecf9e950c9.txt,4117,hard,different also examinations,210.952629,82.830748,"[The young doctor said that I needed two different sets of xrays and also needed more examinations. , The food tasted a lot different but could have also tasted odd because I had dental examinations done earlier.]","[The different results also led the doctor to believe that more examinations were in order over the next few days. , The dogs were all different colors and they all also needed separate flea examinations.]","[But, that was different from her own, because she was also in her early stages of admission examinations., It is no different from the other prayers and also from the other examinations., Different characteristics of the different kinds of positive mental activities are also evident in different subjects, in which it seems that one has to pay attention to one's teacher in particular, as in examinations, because the teacher may not be aware of the objects., The activities of different students are also influenced by different degrees of success, as these examinations are to be one-sided., But this is not a different kind of missionary work, and it is also a more valuable thing to have these examinations as well.]"
10,8951b4f7-be69-47fb-ab0a-280c02451a61.txt,3684,hard,intellect supreme dear,47.129451,313.557370,"[The intellect of the supreme being was something the race held dear., My intellect regarding the topic of a supreme pizza was something nobody thought was dear.]","[Her intellect was supreme above all others that she held dear., He thought his intellect was supreme but his dear sister thought otherwise.]","[Therefore, the Word of God's Wisdom, or the intellect of the Supreme Soul, is dear to the spiritual person., 'Not to you, but for your intellect, supreme, dear friend., You must have a refined intellect, the supreme ego, your dear self, and such not-so-subtle habits as you would think., Ana was of a man of a particular character and her intellect was supreme, dear friend., His intuition and the pure intellect of the Supreme Being are the very main means for creating the righteous, and he knows the expression of justice for a dear cause.]"
100,07b1ecd8-916c-42af-9cb1-81e8ac6809b9.txt,2360,hard,corner his away,100.300951,70.522892,"[Someone had left piles of old furniture on the corner, so he shooed his neighbor's kids away., We walked to the corner so we could say goodbye to his grandmother before she went away.]","[I stapled a sign to the corner of the room to notify his sister we had gone away on a trip., I never knew that the corner store was his home away from home.]","[He peered into the corner of the porch and looked over his shoulder to see his mother stepping away., He would have walked right out of the room, but he was startled when the corner of his door pulled away from the wall., He turned a corner, his eyes dark and pointed, his hair falling away from his face., In the corner of his eye, an old man turned away., In the corner of his eye, I saw the figure jump away.]"
101,07b1ecd8-916c-42af-9cb1-81e8ac6809b9.txt,3863,hard,mental these suggested,41.408520,100.626432,"[I asked the therapist for mental health aids, and 

In [3]:
'''Rater responses for judgment groups'''

responses = pandas.read_json("judgment_group_responses.json")
responses

,Group_ID,Block_ID,Input_Words,Difficulty_Level,Pre_Sentence,Post_Sentence,Generated_Sentence,Prefers_Pre_Sentence,Prefers_Post_Sentence,Prefers_Generated_Sentence,Observed_Generated_Sentences
0,0,1891,wake fallen asleep,easy,The mother yelled at the child to wake up but the child had fallen asleep again in front of the TV.,The dog would wake up the baby with it's loud bark after the baby had fallen asleep.,I wake up to find myself fallen asleep again.,0,1,0,"[I wake up to find myself fallen asleep again., He'd wake the other night and find her had fallen asleep., I wake up in a cold sweat after I have fallen asleep., When I wake up, I've fallen asleep., They would wake the fallen enemy and fall asleep with him.]"
1,0,1891,wake fallen asleep,easy,The mother yelled at the child to wake up but the child had fallen asleep again in front of the TV.,The dog would wake up the baby with it's loud bark after the baby had fallen asleep.,I wake up to find myself fallen asleep again.,0,1,0,"[I wake up to find myself fallen asleep again., He'd wake the other night and find her had fallen asleep., I wake up in a cold sweat after I have fallen asleep., When I wake up, I've fallen asleep., They would wake the fallen enemy and fall asleep with him.]"
2,1,2004,are cancer treatment,easy,There are many people battling cancer that are looking for an alternative treatment.,Many companies are researching cancer in hopes to come up with a milder form of effective treatment.,"You are not a doctor and you are not a patient, so your cancer treatment is just a minor miracle.",0,0,1,"[You are not a doctor and you are not a patient, so your cancer treatment is just a minor miracle., Our findings are currently being validated in an orthotopic pancreatic cancer model for treatment of CRC., Those patients are not eligible for cancer treatment and may die., We are discussing ways to eradicate cancer by introducing treatment strategies., We are currently under investigation and are currently studying the effect of ESR1 inhibition on cancer cell survival, and the mechanism of treatment-mediated apoptosis and apoptosis in vivo.]"
3,1,2004,are cancer treatment,easy,There are many people battling cancer that are looking for an alternative treatment.,Many companies are researching cancer in hopes to come up with a milder form of effective treatment.,"You are not a doctor and you are not a patient, so your cancer treatment is just a minor miracle.",0,1,0,"[You are not a doctor and you are not a patient, so your cancer treatment is just a minor miracle., Our findings are currently being validated in an orthotopic pancreatic cancer model for treatment of CRC., Those patients are not eligible for cancer treatment and may die., We are discussing ways to eradicate cancer by introducing treatment strategies., We are currently under investigation and are currently studying the effect of ESR1 inhibition on cancer cell survival, and the mechanism of treatment-mediated apoptosis and apoptosis in vivo.]"
4,2,208,walking and seeing,easy,The little children enjoyed walking through the zoo and seeing all the different animals.,The woman cried when she saw her little girl walking and seeing for the first time after she got her new glasses.,"She felt the urge to cry, but she kept walking and seeing no sign of it.",1,0,0,"[She felt the urge to cry, but she kept walking and seeing no sign of it., He was walking in front of the stove and he looked down on the ground seeing what was going on., We were walking in and were immediately upon seeing what the neighbors had in store., She was walking with a friend, and she just happened to be seeing a man, a man, and he was going to kill her., She could hear men walking up and down the alley, and she didn't know what they were doing, but she couldn't deny seeing the resemblance.]"
...,...,...,...,...,...,...,...,...,...,...,...
1739,869,771,felt meet again,easy,Jenna felt a spooky sense of deja vu and felt that she was about to meet a familia

### Permutation test for statistical significance

In [4]:
import numpy

def eval_diff(model1_values, model2_values, num_trials=100000, verbose=False):
    '''return p-value of difference between model1_values and model2_values'''

    model1_values = model1_values.flatten()
    model2_values = model2_values.flatten()
    # filter any NaN values
    model1_values = model1_values[~numpy.isnan(model1_values)]
    model2_values = model2_values[~numpy.isnan(model2_values)]
    model_difference = numpy.mean(model2_values) - numpy.mean(model1_values)
    counter = 0
    for trial in range(num_trials):
        # reshuffle all predictions between these 2 models
        values = numpy.concatenate((model1_values, model2_values))
        numpy.random.shuffle(values)
        model1_sample = values[:len(model1_values)]
        model2_sample = values[-len(model2_values):]
        sample_difference = numpy.mean(model2_sample) - numpy.mean(model1_sample)
        if numpy.abs(sample_difference) >= numpy.abs(model_difference):  # two tailed test
            counter += 1
        if verbose and trial % 500 == 0:
            print("completed", trial, "trials...")
    p_val = float(counter + 1) / (num_trials + 1)

    return p_val

### Results

#### Human vs. Generated Storiability (Table 3)

In [5]:
responses[['Prefers_Pre_Sentence',
           'Prefers_Post_Sentence',
           'Prefers_Generated_Sentence']].mean()

Prefers_Pre_Sentence          0.356078
Prefers_Post_Sentence         0.364679
Prefers_Generated_Sentence    0.279243
dtype: float64

In [6]:
eval_diff(responses['Prefers_Pre_Sentence'].values, 
          responses['Prefers_Generated_Sentence'].values)

9.99990000099999e-06

#### Prompt Difficulty and Storiability (Table 5)

In [7]:
responses.groupby('Difficulty_Level')[['Prefers_Pre_Sentence',
           'Prefers_Post_Sentence',
           'Prefers_Generated_Sentence']].mean()

,Prefers_Pre_Sentence,Prefers_Post_Sentence,Prefers_Generated_Sentence
Difficulty_Level,,,
easy,0.384434,0.353774,0.261792
hard,0.329241,0.375000,0.295759


In [8]:
easy_responses = responses[responses['Difficulty_Level'] == 'easy']
hard_responses = responses[responses['Difficulty_Level'] == 'hard']

In [9]:
eval_diff(easy_responses['Prefers_Pre_Sentence'].values, 
          easy_responses['Prefers_Post_Sentence'].values)

0.20982790172098278

In [10]:
eval_diff(hard_responses['Prefers_Pre_Sentence'].values,
          hard_responses['Prefers_Post_Sentence'].values)

0.047239527604723955

##### For other analyses of human-authored sentences, reshape response data into one sentence per row, where each sentence is marked as preferred or not preferred

In [11]:
sentences = {'Item_Num': [],
             'Group_ID': [],
             'Block_ID': [],
             'Sentence': [],
             'Input_Words': [],
             'Difficulty_Level': [],
             'Condition': [],
             'Is_Preferred': [],
             'Observed_Examples': []}

for item_idx, item in responses.iterrows():
    for sent_type in ['Pre', 'Post']:
        pref_column_label = "Prefers_" + sent_type + "_Sentence"
        preferred = 0
        if item[pref_column_label] == 1:
            preferred = 1
        sent_column_label = sent_type + "_Sentence"
        sent = item[sent_column_label]
        sentences['Item_Num'].append(item_idx)
        sentences['Group_ID'].append(item['Group_ID'])
        sentences['Sentence'].append(sent)
        sentences['Block_ID'].append(item['Block_ID'])
        sentences['Input_Words'].append(item['Input_Words'])
        sentences['Difficulty_Level'].append(
            item['Difficulty_Level'])
        sentences['Condition'].append(sent_type)
        sentences['Is_Preferred'].append(preferred)
        sentences['Observed_Examples'].append(
            item['Observed_Generated_Sentences'])

sentences = pandas.DataFrame(sentences)
sentences

,Item_Num,Group_ID,Block_ID,Sentence,Input_Words,Difficulty_Level,Condition,Is_Preferred,Observed_Examples
0,0,0,1891,The mother yelled at the child to wake up but the child had fallen asleep again in front of the TV.,wake fallen asleep,easy,Pre,0,"[I wake up to find myself fallen asleep again., He'd wake the other night and find her had fallen asleep., I wake up in a cold sweat after I have fallen asleep., When I wake up, I've fallen asleep., They would wake the fallen enemy and fall asleep with him.]"
1,0,0,1891,The dog would wake up the baby with it's loud bark after the baby had fallen asleep.,wake fallen asleep,easy,Post,1,"[I wake up to find myself fallen asleep again., He'd wake the other night and find her had fallen asleep., I wake up in a cold sweat after I have fallen asleep., When I wake up, I've fallen asleep., They would wake the fallen enemy and fall asleep with him.]"
2,1,0,1891,The mother yelled at the child to wake up but the child had fallen asleep again in front of the TV.,wake fallen asleep,easy,Pre,0,"[I wake up to find myself fallen asleep again., He'd wake the other night and find her had fallen asleep., I wake up in a cold sweat after I have fallen asleep., When I wake up, I've fallen asleep., They would wake the fallen enemy and fall asleep with him.]"
3,1,0,1891,The dog would wake up the baby with it's loud bark after the baby had fallen asleep.,wake fallen asleep,easy,Post,1,"[I wake up to find myself fallen asleep again., He'd wake the other night and find her had fallen asleep., I wake up in a cold sweat after I have fallen asleep., When I wake up, I've fallen asleep., They would wake the fallen enemy and fall asleep with him.]"
4,2,1,2004,There are many people battling cancer that are looking for an alternative treatment.,are cancer treatment,easy,Pre,0,"[You are not a doctor and you are not a patient, so your cancer treatment is just a minor miracle., Our findings are currently being validated in an orthotopic pancreatic cancer model for treatment of CRC., Those patients are not eligible for cancer treatment and may die., We are discussing ways to eradicate cancer by introducing treatment strategies., We are currently under investigation and are currently studying the effect of ESR1 inhibition on cancer cell survival, and the mechanism of treatment-mediated apoptosis and apoptosis in vivo.]"
...,...,...,...,...,...,...,...,...,...
3483,1741,870,3910,There was a paint smear that dried up in the scouring heat.,was smear heat,hard,Post,0,"[It was a big smear of heat from the sun., I was surprised to see a smear of skin beneath the heat., The material was burnt and a faint smear of red pebble stuck to it, but the heat was nothing to the blood, not even as it gushed from his face., I was pale and trembling with the white and pink, a smear of sun, and my body shivered under the heat., There was a smear of grease and the odor of cooking oil in the heat.]"
3484,1742,871,29,"The lookout scanned the horizon at dusk, as the army expected the enemy to attack at the shore of the bay.",enemy at bay,easy,Pre,1,"[I know I am not enemy but I am not at bay., His enemy was able to get their eyes at bay., The enemy was at the gate, but their weapons were long dead at bay., He wanted to look at the enemy and be able to shoot them at bay., No longer in the enemy's company at bay.]"
3485,1742,871,29,The soldier kept his enemy at bay by brandishing his automatic rifle and hand grenades.,enemy at bay,easy,Post,0,"[I know I am not enemy but I am not at bay., His enemy was able to get their eyes at bay., The enemy was at the gate, but their weapons were long dead at bay., He wanted to look at the enemy and be able to shoot them at bay., No longer in the enemy's company at bay.]"
3486,1743,871,29,"The lookout scanned the horizon at dusk, as the army expected the enemy to attack at the shore of the bay.",enemy at bay,easy,Pre,0,"[I know I am not enemy but I am not at bay., His enemy was able to get their eyes at bay., The e

In [12]:
easy_sentences = sentences[sentences['Difficulty_Level'] == 'easy']
hard_sentences = sentences[sentences['Difficulty_Level'] == 'hard']

##### Number of Infilled Tokens by Difficulty (i.e. validation of difference between easy and hard prompts; Table 4)

In [13]:
'''Compute average number of infilled words between prompt tokens'''

def get_mean_token_distance(input_words, sent):
    sent_tokens = vectorizer.tokenizer.tokenize(sent)
    # Merge subtokens
    sent_tokens = " ".join(sent_tokens).replace(" ##", "").split(" ")
    token_idxs = []
    next_input_word_idx = 0
    for idx, token in enumerate(sent_tokens):
        if token.lower() == input_words[next_input_word_idx]:
            token_idxs.append(idx)
            next_input_word_idx +=1
            if next_input_word_idx == len(input_words):
                break
    dists = []
    for idx1, idx2 in zip(token_idxs[:-1], token_idxs[1:]):
        dists.append(idx2 - idx1 - 1)
    return numpy.mean(dists)

distances = []
for idx, item in sentences.iterrows():
    distance = get_mean_token_distance(item['Input_Words'].split(), 
                                       item['Sentence'])
    distances.append(distance)
    if idx and idx % 1000 == 0:
        print(idx)
        # break
sentences['Num_Infilled_Words'] = distances

1000
2000
3000


In [14]:
sentences.groupby('Difficulty_Level')['Num_Infilled_Words'].mean()

Difficulty_Level
easy    3.035377
hard    4.316964
Name: Num_Infilled_Words, dtype: float64

In [16]:
easy_sentences = sentences[sentences['Difficulty_Level'] == 'easy']
hard_sentences = sentences[sentences['Difficulty_Level'] == 'hard']

In [17]:
eval_diff(easy_sentences['Num_Infilled_Words'].values,
          hard_sentences['Num_Infilled_Words'].values)

9.99990000099999e-06

#### Influence of Generated Examples (Table 6)

In [18]:
'''Compute similarity between human-authored sentences and observed generated examples'''

def get_sent_vector_sim_scores(sent1, sents2):
    sent1_vector = numpy.mean(vectorizer(sent1), axis=1)
    sim_scores = []
    for sent2 in sents2:
        sent2_vector = numpy.mean(vectorizer(sent2), axis=1)
        sim_score = cosine_similarity(sent1_vector, 
                                      sent2_vector)[0][0]
        sim_scores.append(sim_score)
    return sim_scores
    
def get_sent_vector_sim_to_gen(sent, gen_sents):#, reduce='mean'):
    score = numpy.max(get_sent_vector_sim_scores(sent, gen_sents))#, reduce=reduce))
    return score

scores = []

for idx, item in sentences.iterrows():
    score = get_sent_vector_sim_to_gen(
        item['Sentence'],
        item['Observed_Examples'])
    scores.append(score)

    if idx and idx % 1000 == 0:
        print(idx)

sentences['Sent_Gen_Sim'] = scores

1000
2000
3000


In [19]:
hard_sentences = sentences[sentences['Difficulty_Level'] == 'hard']

In [20]:
hard_sentences.groupby('Condition')['Sent_Gen_Sim'].mean()

Condition
Post    0.923384
Pre     0.920936
Name: Sent_Gen_Sim, dtype: float64

In [21]:
hard_post_sentences = hard_sentences[hard_sentences['Condition'] == 'Post']
hard_pre_sentences = hard_sentences[hard_sentences['Condition'] == 'Pre']

In [22]:
eval_diff(hard_pre_sentences['Sent_Gen_Sim'].values,
          hard_post_sentences['Sent_Gen_Sim'].values)

0.0048599514004859954

#### Influence and Storiability (Table 7)

In [23]:
hard_post_sentences.groupby('Is_Preferred')['Sent_Gen_Sim'].mean()

Is_Preferred
0    0.922314
1    0.925166
Name: Sent_Gen_Sim, dtype: float64

In [24]:
hard_post_preferred_sentences = hard_post_sentences[hard_post_sentences['Is_Preferred'] == 1]
hard_post_notpreferred_sentences = hard_post_sentences[hard_post_sentences['Is_Preferred'] == 0]


In [25]:
eval_diff(hard_post_preferred_sentences['Sent_Gen_Sim'].values,
          hard_post_notpreferred_sentences['Sent_Gen_Sim'].values)

0.017409825901740984